In [11]:
import pandas as pd
from datetime import datetime as dt
import yfinance as yf
import numpy as np

assets = ['AAPL', 'TSLA', 'NVDA', 'GOOGL','MSFT','AMZN','META']
benchmarks = ['SPY','IWM','DIA']

#Date times
#Date functions found on StackOverflow
end_date = dt.now().strftime('%Y-%m-%d')
startdate_3m = pd.to_datetime(end_date) - pd.DateOffset(months = 3)
startdate_12m = pd.to_datetime(end_date) - pd.DateOffset(months = 12)

#3months, 12months, benchmark data
data_3m = yf.download(tickers = assets, start = startdate_3m, end = end_date, auto_adjust=True)['Close']
data_12m = yf.download(tickers = assets, start = startdate_12m, end = end_date, auto_adjust=True)['Close']
benchmarks_data = yf.download(tickers = benchmarks, start = startdate_12m, end = end_date)['Close']

#Portfolio weight
equal_weight = 1 / len(assets)

#Daily returns for volatiltity
returns_3m = data_3m.pct_change().dropna()
returns_12m = data_12m.pct_change().dropna()
returns_benchmark = benchmarks_data.pct_change().dropna()

#Annualized volatility - formula (Standard Deviation * square root of 252)
Annualized_Volatility = returns_3m.std() * np.sqrt(252) 

#StackOverflow used to find the covariance and variance snippets for the beta
#Defined function to find beta by calculating covariance and variance
def calculate_beta(asset_returns, benchmark_returns):
    cov_matrix = np.cov(asset_returns, benchmark_returns)
    cov_stock_benchmark = cov_matrix[0, 1]
    var_benchmark = np.var(benchmark_returns)
    beta = cov_stock_benchmark / var_benchmark
    return beta

#Calculates the beta against the benchmarks
betas_spy = [calculate_beta(returns_12m[ticker], returns_benchmark['SPY']) for ticker in assets]
betas_iwm = [calculate_beta(returns_12m[ticker], returns_benchmark['IWM']) for ticker in assets]
betas_dia = [calculate_beta(returns_12m[ticker], returns_benchmark['DIA']) for ticker in assets]

#Risk analysis table
analysis = pd.DataFrame({
    'Portfolio Weight' : equal_weight,
    'Annualized Volatility (3 months)' : Annualized_Volatility,
    'Beta against SPY' : betas_spy,
    'Beta against IWM' : betas_iwm,
    'Beta against DIA' : betas_dia
})

print (analysis)

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  3 of 3 completed


        Portfolio Weight  Annualized Volatility (3 months)  Beta against SPY  \
Ticker                                                                         
AAPL            0.142857                          0.245898          0.918515   
AMZN            0.142857                          0.262390          2.628262   
GOOGL           0.142857                          0.340335          2.840861   
META            0.142857                          0.278130          1.243297   
MSFT            0.142857                          0.239582          1.164601   
NVDA            0.142857                          0.601670          1.492189   
TSLA            0.142857                          0.615895          1.281646   

        Beta against IWM  Beta against DIA  
Ticker                                      
AAPL            0.327913          0.684975  
AMZN            1.311529          1.930109  
GOOGL           0.892199          1.134976  
META            0.491281          0.789601  
MSFT     